In [97]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import string
import re
import os
import time
import pickle
import tensorflow as tf
import en_core_web_sm
import tensorflow_hub as hub
import nltk
import nltk.data
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import line_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import tree2conlltags
from sklearn.model_selection import train_test_split
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import PorterStemmer
from nltk.stem.porter import *
from gensim import corpora, models
from wordcloud import WordCloud, STOPWORDS
import nltk, math, codecs
from gensim.models import Doc2Vec
from nltk.cluster.kmeans import KMeansClusterer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
pd.set_option('display.max_colwidth', 200)

In [64]:
import os
os.getcwd()

'/sandeep/Offer_extraction'

In [3]:
print(tf.__version__)

1.14.0


In [4]:
#Set working directory
main_dir ='/sandeep/Offer_extraction/Audio_Transcription-dev/Amazon/Transcripts2/'
os.chdir(main_dir)

In [5]:
location = os.getcwd() # get present working directory location here

df_list=[]

for file in os.listdir(location):
    if file.endswith(".txt"):
        df = nltk.data.load(file)
        df_list.append(df)

In [33]:
df = pd.DataFrame(df_list, index=None)
df.columns = ['chats']

In [34]:
df.head()

,chats
0,"What? I mean, when the status, That's what I would recommend. Telecom. Just turn keeping for Mary Orlando. I was see anyone when I saw the movie star Atlanta come up in the winter? Well, yeah, I t..."
1,"So they're calling Comcast. My name is Kiera. How may I help you today? Yes. Uh, I just got my box in the mail and I'm currently trying to install it. I've got it to the point where it says welcom..."
2,"Hi. Thank you for choosing Comcast. This is very how can I help you, Marie? Okay. By my my paying for, uh, premium channels since December and we got no no premium channels. It says subscription s..."
3,"Hello. Thank you so much for calling Comcast. This is Chris, and why? I was getting a technical support. How may I help you? It was Bill. Normally, I only the little sales people call me about t..."
4,"Hi. Did they thank you for calling Xfinity. And my name is Karlene. And how may I assist you? Yes. I like at explanation as to why my bill increase over twenty six dollars. Okay, so, um, I'll be m..."


In [35]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

df['chats'] = df['chats'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

In [82]:
!python -m spacy download en
!python -m spacy download en_core_web_sm

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.5/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.5/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [37]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [38]:
df['chats'] = lemmatization(df['chats'])

In [57]:
df.head()

,chats
0,"what -PRON- mean , when the status , that be what -PRON- would recommend . telecom . just turn keep for Mary Orlando . -PRON- be see anyone when -PRON- see the movie star Atlanta come up in the wi..."
1,"so -PRON- be call Comcast . -PRON- name be Kiera . how may -PRON- help -PRON- today yes . uh , -PRON- just get -PRON- box in the mail and -PRON- be currently try to install -PRON- . -PRON- have ge..."
2,"hi . thank -PRON- for choose Comcast . this be very how can -PRON- help -PRON- , Marie okay . by -PRON- -PRON- paying for , uh , premium channel since December and -PRON- get no no premium channel..."
3,"hello . thank -PRON- so much for call Comcast . this be Chris , and why -PRON- be get a technical support . how may -PRON- help -PRON- -PRON- be Bill . normally , -PRON- only the little sale pe..."
4,hi . do -PRON- thank -PRON- for call Xfinity . and -PRON- name be Karlene . and how may -PRON- assist -PRON- yes . -PRON- like at explanation as to why -PRON- bill increase over twenty six dollar ...


In [13]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [75]:
# Extracting ELMo features prototype
x = ["Its nice weather in Philadelphia"]
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
embeddings.shape
print(embeddings)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Tensor("module_apply_default_6/aggregation/mul_3:0", shape=(1, 5, 1024), dtype=float32)


In [15]:
def elmo_vectors(x):
      embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

      with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [16]:
list_df = [df[i:i+100] for i in range(0,df.shape[0],100)]

In [66]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['target']) for x in list_df]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [190]:
elmo_train


[array([[ 0.00298428, -0.05399751,  0.05601773, ...,  0.01693193,
          0.01717028,  0.00312184],
        [ 0.04747167, -0.04471469,  0.04991376, ..., -0.0018085 ,
          0.01849449,  0.02375721],
        [ 0.04258014, -0.05477962,  0.04828866, ...,  0.0089344 ,
          0.01001406,  0.03670227],
        ...,
        [ 0.04736235, -0.04301926,  0.04236198, ..., -0.00366713,
         -0.00922094,  0.02014527],
        [ 0.03664199, -0.04772153,  0.04539322, ..., -0.00259543,
         -0.00432832,  0.02027351],
        [ 0.07486328, -0.04783226,  0.05648524, ..., -0.02577193,
          0.02131773,  0.04982422]], dtype=float32)]

In [191]:
elmo_train_new = np.concatenate(elmo_train, axis=0)

In [192]:
print(elmo_train_new)

[[ 0.00298428 -0.05399751  0.05601773 ...  0.01693193  0.01717028
   0.00312184]
 [ 0.04747167 -0.04471469  0.04991376 ... -0.0018085   0.01849449
   0.02375721]
 [ 0.04258014 -0.05477962  0.04828866 ...  0.0089344   0.01001406
   0.03670227]
 ...
 [ 0.04736235 -0.04301926  0.04236198 ... -0.00366713 -0.00922094
   0.02014527]
 [ 0.03664199 -0.04772153  0.04539322 ... -0.00259543 -0.00432832
   0.02027351]
 [ 0.07486328 -0.04783226  0.05648524 ... -0.02577193  0.02131773
   0.04982422]]


In [29]:
# save elmo_train
pickle_out = open("elmo_train_0919.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

In [58]:
pickle_in = open("elmo_train_0919.pickle","rb")
elmo_train_new = pickle.load(pickle_in)

In [193]:
final_elmo_vectors = pd.DataFrame(data=elmo_train_new)
final_elmo_vectors

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.002984,-0.053998,0.056018,-0.034273,0.003704,-0.004036,-0.003177,0.002186,-0.066305,-0.095743,...,-0.019374,-0.032543,0.036729,0.079802,0.102829,0.046919,0.047411,0.016932,0.017170,0.003122
1,0.047472,-0.044715,0.049914,-0.037812,0.019326,0.003511,-0.045109,0.025959,-0.092643,-0.091779,...,-0.014966,-0.017134,0.028860,0.062348,0.134542,0.063503,0.087361,-0.001809,0.018494,0.023757
2,0.042580,-0.054780,0.048289,-0.037035,0.027517,0.024342,-0.014615,0.025050,-0.067115,-0.087230,...,-0.025296,-0.025354,0.031925,0.045624,0.125164,0.046877,0.055976,0.008934,0.010014,0.036702
3,0.003881,-0.050993,0.047906,-0.026324,0.004444,0.004585,0.001779,0.013547,-0.072063,-0.104591,...,-0.029142,-0.031847,0.029508,0.074330,0.102445,0.034096,0.053981,0.008539,0.004612,0.002290
4,0.034771,-0.043803,0.040984,-0.045625,0.020423,-0.002480,-0.031445,0.055437,-0.079738,-0.082044,...,-0.024629,-0.014863,0.036979,0.030690,0.115476,0.043382,0.097408,-0.009731,-0.006117,0.028270
5,0.031361,-0.048398,0.058708,-0.041302,0.024156,-0.006075,-0.057171,0.022469,-0.096082,-0.079823,...,-0.005115,-0.028225,0.042479,0.044204,0.147780,0.050757,0.098029,-0.009400,0.004692,0.017234
6,0.292129,-0.080455,0.129113,-0.028264,0.095801,0.022306,-0.225429,0.085675,-0.137053,0.034626,...,0.012486,0.020176,0.006401,-0.093407,0.332406,0.082059,0.179962,-0.073717,0.175380,0.163451
7,0.005043,-0.047443,0.059971,-0.024417,0.008742,0.008770,-0.013902,0.007465,-0.079064,-0.092170,...,-0.013045,-0.031069,0.033851,0.078443,0.117725,0.044493,0.057480,0.009285,0.027633,0.000060
8,-0.015358,-0.039511,0.040222,-0.022427,-0.006896,0.003019,0.021765,0.006059,-0.057029,-0.105654,...,-0.035049,-0.029742,0.038843,0.084521,0.076592,0.035268,0.049488,0.018356,-0.011694,-0.012719
9,0.003669,-0.046605,0.049084,-0.016839,0.003198,-0.019879,-0.022834,0.004721,-0.096332,-0.095445,...,-0.030259,-0.029511,0.032621,0.065017,0.104009,0.050994,0.076624,0.001243,-0.001583,-0.001749


In [60]:
final_elmo_vectors.to_csv("final_elmo_vectors.csv")

In [77]:
#nltk.download('punkt')
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [85]:
# Set globals


# Need lower threshold that represents cheapest package. 
# Necessary to filter out noise from phone numbers, quantifiers, etc.
lower_threshold = 15

# Need upper threshold to help rule out discussion of wifi speeds. 
upper_threshold = 250

units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
    ]

tens = ["", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

scales = ["hundred", "thousand", "million", "billion", "trillion"]

no_package_moce_price = {'NO PRODUCTS': 4.99}

single_package_mode_price = {'VIDEO ONLY': 49.99, 'CDV ONLY': 29.99, 'XM ONLY': 30.00, 'HSD ONLY': 39.99
                            , 'XH ONLY' : 24.99
    
}

double_package_mode_price = {'HSD/XH': 39.99 , 'HSD/CDV': 59.99 , 'CDV/XH': 19.99, 'XH/XM': 24.99,
                            'VIDEO/CDV': 74.99 , 'CDV/XM': 25.00, 'VIDEO/XM': 54.99, 'HSD/XM': 39.99 , 
                            'VIDEO/XH': 29.99 , 'VIDEO/HSD': 49.99 
}

triple_package_mode_price = {'VIDEO/CDV/XH': 24.99 , 'VIDEO/HSD/XM': 89.99 , 'HSD/CDV/XM': 59.99 ,
                            'CDV/XH/XM': 24.10 , 'VIDEO/HSD/XH': 109.99 , 'HSD/XH/XM': 39.99 , 
                            'VIDEO/HSD/CDV': 119.99 , 'HSD/CDV/XH': 59.99 , 
                            'VIDEO/CDV/XM': 54.99, 'VIDEO/XH/XM': 24.99
                            
    
}

quad_package_mode_price = {'VIDEO/HSD/XH/XM': 109.99, 'VIDEO/HSD/CDV/XH': 24.99, 'HSD/CDV/XH/XM': 29.95, 
                          'VIDEO/CDV/XH/XM': 24.99, 'VIDEO/HSD/CDV/XM': 119.99
    
}

price_dict = dict(single_package_mode_price)
price_dict.update(double_package_mode_price) 
price_dict.update(triple_package_mode_price) 
price_dict.update(quad_package_mode_price) 

master_price_dict = {19.99: 'CDV/XH',
               24.10: 'CDV/XH/XM', 
               24.99: ('XH ONLY', 'XH/XM', 'VIDEO/CDV/XH', 'VIDEO/XH/XM',
                       'VIDEO/HSD/CDV/XH', 'VIDEO/CDV/XH/XM', 'CDV/XM'), 
                29.99: ('HSD/CDV/XH/XM', 'CDV ONLY', 'VIDEO/XH', 'XM ONLY'), 
               39.99: ('HSD ONLY', 'HSD/XH', 'HSD/XM', 'HSD/XH/XM'), 
               49.99: ('VIDEO ONLY', 'VIDEO/HSD'), 
               54.99: ('VIDEO/XM', 'VIDEO/CDV/XM'), 
               59.99: ('HSD/CDV', 'HSD/CDV/XM', 'HSD/CDV/XH'), 
               74.99: 'VIDEO/CDV', 
               89.99: 'VIDEO/HSD/XM', 
               109.99: ('VIDEO/HSD/XH', 'VIDEO/HSD/XH/XM'), 
               119.99: ('VIDEO/HSD/CDV', 'VIDEO/HSD/CDV/XM')
              }
               


# price_dict_rev = {v: k for k, v in price_dict.items()}
price_list = [19.99, 24.10, 24.99, 29.99, 39.99, 49.99, 54.99, 59.99, 74.99, 89.99, 109.99, 119.99]


text_transformation_dic={'home security system':'home-security-system',\
     'security system':'security-system',\
     'home security':'home-security',
     'set top box':'set-top-box',\
     'xfinity mobile':'xfinity-mobile',\
     'infinity mobile':'xfinity-mobile',\
     'land line':'landline',\
     'home phone':'home-phone',\
     'hot spot':'hotspot',\
     'home phones':'home-phone',\
     'quad play':'quad-play',\
     'double play':'double-play',\
     'triple play':'triple-play',\
     'single play':'single-play',\
     'performance pro':'performance-pro',\
     'performance plus':'performance-plus',\
     'performance starter':'performance-starter',\
     'amazon prime': 'amazon-prime',\
     'sling tv':'sling-tv',\
     'direct tv':'direct-tv',\
     'high definition':'high-definition'
    }


# Negative keywords to look for in text.
estimates = ['about', 'around', 'almost', 'nearly', 'between', 'approaching']
post_num = ['bytes', 'byte', 'megabytes', 'megabyte', 'gigabytes', 'gigabyte', 
            'terabytes', 'terabyte', 'channels', 'channel', 'charge', 'credit']

# Positive keywords to look for in text
key_num1 = 'ninety nine'
key_num2 = 'ninety five' 
key_words1 = ['package', 'promotion', 'offer', 'offering', 'bundle', 'bundles', 'month']
    
# Specific tier keywords. 
hsd=['speed','internet','megabyte','megabytes','megabits','modem','gigabytes','signal','wifi','network','router']
video=['tv','TV','channel','channels','set-top-box','dvr','hd','hbo','espn','box','boxes']
cdv=['landline','home-phone']
xm=['mobile','lines','hotspot','spot','xfinity-mobile']
xh=['security-system','home-security-system','home-security-system','home-security','security','camera','cameras']

# Other Categories of keywords
stream=['streaming','stream']
ott=['netflix','hulu','roku']
hsd_tier=['blast','performance','performance-pro','performance-plus','performance-starter','gigabit']
vid_tier=['latino','dvr','hd','high-definition']
packages=['quad-play','double-play','triple-play','single-play']

nlp = en_core_web_sm.load()

In [86]:
def replace_all(text, dic):
    for i, j in dic.items():
        src_str  = re.compile(i, re.IGNORECASE)
        text = src_str.sub(j,text)
    return text


In [87]:
# Converts text form of number to decimal.

def text2int(textnum, numwords={}):
    
    # Put everything in lower case, strip commas, and call missing_category_check on textnum. 
    textnum = textnum.lower()
    textnum = textnum.replace(',', '')
    textnum = missing_category_check(textnum)

    # Made True upon finding keywords from 'descriptors'
    take_ave = False 
    waiting_for_and = False
    
    # Stores values for calculating averages with 'to' and 'between' cases.
    keep = [] 

    # List of non number keywords to look for.
    descriptors = ['a', 'to', 'between','dollars', 'cents']
      
    if not numwords:      
    # And symbolizes addition
      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):
          numwords[word] = (1, idx)
      for idx, word in enumerate(tens):
          numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):  
          numwords[word] = (10 ** (idx * 3 or 2), 0)
        
    current = result = 0
    
    # For each word test if in 'descriptors or if a number word.'
    for word in textnum.split():
        if word not in numwords and word not in descriptors:
            continue 
        # 'a' in a number clause symbolizes '1'
        if word == 'a': 
            word = 'one' 
        # 'to' compares two values. When found will calculate average of two. 
        elif word == 'to': 
            take_ave = True 
            keep.append(current) 
            current = 0
            result = 0
            continue
        # 'between' compares two values in conjunction with 'and'. When found will calculate average of two. 
        elif word == 'between':
            waiting_for_and = True
            continue
        elif waiting_for_and and word == "and":
            waiting_for_and = False
            take_ave = True
            keep.append(current)
            current = 0
            result = 0
            continue
        # Anything after the word dollars needs to be treated as cents. 
        # Missing category_check will add cents to end on proper condition if speaker hasn't.  
        elif word == 'dollars':
            result = result + current 
            current = 0
            continue
        # Makes sure cents aren't being read as dollars. 
        elif word == 'cents':
            current = current * .01
            continue

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0
            
    # if 'to' or 'between' was found in clause, calculate average. Otherwise proceed normally.        
    if take_ave and keep[0] != 0: 
        return(result + current + sum(keep)) / (len(keep) + 1)
    else:
        return result + current


In [88]:
# Takes account for cases where speaker is speaking improperly including. 
# 1.) Not say hundred after give hundreds place 
# 2.) Saying something like thirty nine ninety nine rather than thiry nine dollars and ninety nine cents

def missing_category_check(textnum):
    change_made = False
    cent_added = False
    prev_word = ''
    count = 0
    words = textnum.split()
    for word in words:
        if prev_word in units and word in tens and count == 1:
            word = 'hundred ' + word
            change_made = True
            words[count] = word
        elif (prev_word in units or prev_word in tens) and word in tens and count > 1:
            word = 'dollars and ' + word
            cent_added = True
            change_made = True
            words[count] = word
        prev_word = word
        count += 1

    if change_made:
        s = " ".join(words)
        if cent_added:
            s = s + " cents"
        return s
    else:
        return textnum

In [89]:
# Determines how many sentences before and after the current one we can take in for context. 

def check_position(i, sentence_count):
    if i > 1:
        low = i - 2
    else: 
        low = 0
    if sentence_count - 1 - i > 1:
        high = i + 3 
    else: 
        high = sentence_count
    return([low, high])

In [90]:
# importance_analysis(context, X.text) 
# Shows on a scale of one to ten how important this price is. 

def price_importance_analysis(context, token): 
    # Removes all punctuation from context except for hyphens and token and splits into array.
    remove = string.punctuation.replace("-", "")
    context = context.lower().translate(str.maketrans('', '', remove))
    token = token.lower().translate(str.maketrans('', '', remove))
    
    # Default score is 0. 
    score = 0
    
    # Booleans to make sure each category only gets accounted for once. 
    estimate_found = False
    post_num_found = False
    key_words1_found = False 
    packages_found = False 
    
    # Look in context and token for above words. Token has more importance than context. 
    for word in context.split(): 
        if word in estimates and not estimate_found:
            if word in token: 
                score -= 2
            else: 
                score -= 1 
            estimate_found = True 
            continue
        if word in post_num and not post_num_found:
            if word in token: 
                score -= 2
            else: 
                score -= 1
            post_num_found = True
            continue 
        if word in key_words1 and not key_words1_found: 
            score += 2
            key_words1_found = True
            continue
        if word in packages and not packages_found:
            score += 2 
            packages_found = True
            continue

    # Ninety nine and ninety five are big indicators of package offerings. 
    if key_num1 in token or key_num2 in token:
        score += 4 
    elif key_num1 in context or key_num2 in context: 
        score += 2
    
    # Commas in token imply list of numbers. 
    if ',' in token: 
        score -= 3
    

    return score


# price_importance_analysis("IT'S A HUNDRED AND TWENTY DOLLARS A MONTH, UM, FOR THE TRIPLE-PLAY AND LET ME SEE ONE MORE THING", "a hundred and twenty")|

In [91]:
# Forms guess based on price if score is above 2. 
def make_guess(price):
    
    # Finds closest value in price list to given price. 
    closest_num = min(price_list, key=lambda x:abs(x-price))
    
    # Returns packages that are most commonly priced here. 
    return master_price_dict[closest_num]

In [92]:
# Want only Cardinal and Money types. Gives context of two sentences before and after
# Don't want lists of numbers. Ignore tokens that are lists of number. 
# Capitalizes sentence that had mention in it. 

def extract_info(transcript_sentences):
    i = 0 
    guess = ''
    max_score = -10
    transcript_price = 0 
    transcript_score = 0 
    transcript_context = ''
    transcript_label = []
    transcript_score = 0 
    transcript_context = '' 
    sentence_count = len(transcript_sentences)
    while i < sentence_count:
        context_gauge = check_position(i, sentence_count) 
        lower_limit = context_gauge[0] 
        upper_limit = context_gauge[1]
        doc = nlp(transcript_sentences[i])
        
        for X in doc.ents: 
            # When comma inside X.text this implies list of numbers. Don't want these. 
            if X.label_ in ('CARDINAL', 'MONEY', 'QUANTITY') and X.text.count(',') < 1: 
                price = text2int(X.text)
                if price > lower_threshold and price < upper_threshold: 
                    score = price_importance_analysis(transcript_sentences[i], X.text)
                    context = [transcript_sentences[lower_limit:i], transcript_sentences[i].upper(), transcript_sentences[i+1:upper_limit]]
                    if score >= max_score: 
                        max_score = score 
                        transcript_price = price
                        transcript_score = score 
                        transcript_context = context
                        # transcript_context = transcript_sentences[i]
        i += 1
    if transcript_score > 3: 
        guess = make_guess(transcript_price) 
    else: 
        guess = 'NONE' 
    return [transcript_score, transcript_price, transcript_context, guess]

In [95]:
def main():
    rows_list = [] 
    error_count = 0 
    audio_directory = "/sandeep/Offer_extraction/Audio_Transcription-dev/Amazon/Transcripts2/"
    for filename in os.listdir(audio_directory): 
        if filename.endswith(".txt"):
            transcript_file_path = audio_directory + "/" + filename
            transcript_file = open(transcript_file_path, "r")
            transcript_contents = transcript_file.read().lower()
            transcript_contents = replace_all(transcript_contents, text_transformation_dic)
            doc = nlp(transcript_contents)
            transcript_sentences = tokenizer.tokenize(transcript_contents) 
            data = extract_info(transcript_sentences)  
            rows_list.append(data)
            transcript_file.close() 
    df = pd.DataFrame(rows_list)
    df.columns = ['Score', 'Price', 'Context', 'Guess']
    return df

In [98]:
df = main()

In [194]:
df

,Score,Price,Context,Guess
0,0,40.00,"[[and you can select units, you know, make people to fill up soon to present telephone number., but i thought you said you didn't connect.], OKAY, MHM FIFTY MAY., [we just got a lot of times someo...",NONE
1,2,60.00,"[[the only thing is because there's the self insulation., will have to get a installation technician out there, which is seven.], I MEAN, YEAH, SEVENTY DOLLARS TOWARDS THE NEXT MONTH., [billing., ...",NONE
2,4,89.00,"[[four., and that includes that will include, um, actually oh, channels and also stars and show time and that's it.], FOUR, FOUR HUNDRED FIFTY FIVE DOLLARS, UH, FOR A HUNDRED AND FIFTY FOUR DOLLAR...",VIDEO/HSD/XM
3,1,19.00,"[[she's looking., picks it.], SHE SAID I'LL GIVE YOU A HUNDRED MEGABYTES PLUS FOR THIRTY NINE BECAUSE I HAVE TWENTY NINE DOLLARS A MONTH NOW., [not okay., she said she would work in about twenty m...",NONE
4,4,19.89,"[[thirty., um, so it's just a penny off from when you had the original rate of one.], TWENTY NINE NINETY NINE., [um, let me just are you on paperless billing and not okay, let me just see here., n...",CDV/XH
5,6,114.89,"[[okay, um, customer first, um, actually, we would like to pay the bills, but we told her you found out that the promotion provided the word i'm for him last december, twenty fourth of two thousan...","(VIDEO/HSD/XH, VIDEO/HSD/XH/XM)"
6,6,89.00,"[[because they offered quite a bit of stuff too and, uh, to get used to keep me with contest., what could you do for me?], KEEP ME IN THE SAME PACKAGE AT A BETTER PRICE IF I WAS TO ADD M O B. I LO...",VIDEO/HSD/XM
7,0,100.00,"[[you know what?, you you're not going to be able to turn.], GOD DOESN'T HAVE A LOT OF HAVING A HUNDRED., [i'm nobody's kick out., i mean, total romance is something you heard of any other than my...",NONE
8,0,0.00,,NONE
9,0,24.00,"[[and i don't get it, okay?, i was just trying to do read their policy.], SO FOR YOU TO REGISTER THAT YOU HAVE TO LOOK INTO YOUR JACKLEY BROWN EMAIL EXACTLY A DECADE BROWN THIRTY FOUR., [i'm here ...",NONE


In [195]:
final_df_with_elmo = pd.concat([final_elmo_vectors, df], axis=1)
final_df_with_elmo

,0,1,2,3,4,5,6,7,8,9,...,1018,1019,1020,1021,1022,1023,Score,Price,Context,Guess
0,0.002984,-0.053998,0.056018,-0.034273,0.003704,-0.004036,-0.003177,0.002186,-0.066305,-0.095743,...,0.102829,0.046919,0.047411,0.016932,0.017170,0.003122,0,40.00,"[[and you can select units, you know, make people to fill up soon to present telephone number., but i thought you said you didn't connect.], OKAY, MHM FIFTY MAY., [we just got a lot of times someo...",NONE
1,0.047472,-0.044715,0.049914,-0.037812,0.019326,0.003511,-0.045109,0.025959,-0.092643,-0.091779,...,0.134542,0.063503,0.087361,-0.001809,0.018494,0.023757,2,60.00,"[[the only thing is because there's the self insulation., will have to get a installation technician out there, which is seven.], I MEAN, YEAH, SEVENTY DOLLARS TOWARDS THE NEXT MONTH., [billing., ...",NONE
2,0.042580,-0.054780,0.048289,-0.037035,0.027517,0.024342,-0.014615,0.025050,-0.067115,-0.087230,...,0.125164,0.046877,0.055976,0.008934,0.010014,0.036702,4,89.00,"[[four., and that includes that will include, um, actually oh, channels and also stars and show time and that's it.], FOUR, FOUR HUNDRED FIFTY FIVE DOLLARS, UH, FOR A HUNDRED AND FIFTY FOUR DOLLAR...",VIDEO/HSD/XM
3,0.003881,-0.050993,0.047906,-0.026324,0.004444,0.004585,0.001779,0.013547,-0.072063,-0.104591,...,0.102445,0.034096,0.053981,0.008539,0.004612,0.002290,1,19.00,"[[she's looking., picks it.], SHE SAID I'LL GIVE YOU A HUNDRED MEGABYTES PLUS FOR THIRTY NINE BECAUSE I HAVE TWENTY NINE DOLLARS A MONTH NOW., [not okay., she said she would work in about twenty m...",NONE
4,0.034771,-0.043803,0.040984,-0.045625,0.020423,-0.002480,-0.031445,0.055437,-0.079738,-0.082044,...,0.115476,0.043382,0.097408,-0.009731,-0.006117,0.028270,4,19.89,"[[thirty., um, so it's just a penny off from when you had the original rate of one.], TWENTY NINE NINETY NINE., [um, let me just are you on paperless billing and not okay, let me just see here., n...",CDV/XH
5,0.031361,-0.048398,0.058708,-0.041302,0.024156,-0.006075,-0.057171,0.022469,-0.096082,-0.079823,...,0.147780,0.050757,0.098029,-0.009400,0.004692,0.017234,6,114.89,"[[okay, um, customer first, um, actually, we would like to pay the bills, but we told her you found out that the promotion provided the word i'm for him last december, twenty fourth of two thousan...","(VIDEO/HSD/XH, VIDEO/HSD/XH/XM)"
6,0.292129,-0.080455,0.129113,-0.028264,0.095801,0.022306,-0.225429,0.085675,-0.137053,0.034626,...,0.332406,0.082059,0.179962,-0.073717,0.175380,0.163451,6,89.00,"[[because they offered quite a bit of stuff too and, uh, to get used to keep me with contest., what could you do for me?], KEEP ME IN THE SAME PACKAGE AT A BETTER PRICE IF I WAS TO ADD M O B. I LO...",VIDEO/HSD/XM
7,0.005043,-0.047443,0.059971,-0.024417,0.008742,0.008770,-0.013902,0.007465,-0.079064,-0.092170,...,0.117725,0.044493,0.057480,0.009285,0.027633,0.000060,0,100.00,"[[you know what?, you you're not going to be able to turn.], GOD DOESN'T HAVE A LOT OF HAVING A HUNDRED., [i'm nobody's kick out., i mean, total romance is something you heard of any other than my...",NONE
8,-0.015358,-0.039511,0.040222,-0.022427,-0.006896,0.003019,0.021765,0.006059,-0.057029,-0.105654,...,0.076592,0.035268,0.049488,0.018356,-0.011694,-0.012719,0,0.00,,NONE
9,0.003669,-0.046605,0.049084,-0.016839,0.003198,-0.019879,-0.022834,0.004721,-0.096332,-0.095445,...,0.104009,0.050994,0.076624,0.001243,-0.001583,-0.001749,0,24.00,"[[and i don't get it, okay?, i was just trying to do read their policy.], SO FOR YOU TO REGISTER THAT YOU HAVE TO LOOK INTO YOUR JACKLEY BROWN EMAIL EXACTLY A DECADE BROWN THIRTY FOUR., [i'm here ...",NONE


In [196]:
from sklearn.model_selection import train_test_split

In [197]:
def ifef(col):
    col = str(col)
    if col.startswith('V'):
        return '1'
    if col.startswith('H'):
        return '1'
    if col.startswith('N'):
        return '0'
    if col.startswith('C'):
        return '1'
    if col.startswith('('):
        return '1'
final_df_with_elmo['label'] = np.array(final_df_with_elmo['Guess'].apply(ifef), dtype=int, order='C')

In [198]:
final_df_with_elmo

,0,1,2,3,4,5,6,7,8,9,...,1019,1020,1021,1022,1023,Score,Price,Context,Guess,label
0,0.002984,-0.053998,0.056018,-0.034273,0.003704,-0.004036,-0.003177,0.002186,-0.066305,-0.095743,...,0.046919,0.047411,0.016932,0.017170,0.003122,0,40.00,"[[and you can select units, you know, make people to fill up soon to present telephone number., but i thought you said you didn't connect.], OKAY, MHM FIFTY MAY., [we just got a lot of times someo...",NONE,0
1,0.047472,-0.044715,0.049914,-0.037812,0.019326,0.003511,-0.045109,0.025959,-0.092643,-0.091779,...,0.063503,0.087361,-0.001809,0.018494,0.023757,2,60.00,"[[the only thing is because there's the self insulation., will have to get a installation technician out there, which is seven.], I MEAN, YEAH, SEVENTY DOLLARS TOWARDS THE NEXT MONTH., [billing., ...",NONE,0
2,0.042580,-0.054780,0.048289,-0.037035,0.027517,0.024342,-0.014615,0.025050,-0.067115,-0.087230,...,0.046877,0.055976,0.008934,0.010014,0.036702,4,89.00,"[[four., and that includes that will include, um, actually oh, channels and also stars and show time and that's it.], FOUR, FOUR HUNDRED FIFTY FIVE DOLLARS, UH, FOR A HUNDRED AND FIFTY FOUR DOLLAR...",VIDEO/HSD/XM,1
3,0.003881,-0.050993,0.047906,-0.026324,0.004444,0.004585,0.001779,0.013547,-0.072063,-0.104591,...,0.034096,0.053981,0.008539,0.004612,0.002290,1,19.00,"[[she's looking., picks it.], SHE SAID I'LL GIVE YOU A HUNDRED MEGABYTES PLUS FOR THIRTY NINE BECAUSE I HAVE TWENTY NINE DOLLARS A MONTH NOW., [not okay., she said she would work in about twenty m...",NONE,0
4,0.034771,-0.043803,0.040984,-0.045625,0.020423,-0.002480,-0.031445,0.055437,-0.079738,-0.082044,...,0.043382,0.097408,-0.009731,-0.006117,0.028270,4,19.89,"[[thirty., um, so it's just a penny off from when you had the original rate of one.], TWENTY NINE NINETY NINE., [um, let me just are you on paperless billing and not okay, let me just see here., n...",CDV/XH,1
5,0.031361,-0.048398,0.058708,-0.041302,0.024156,-0.006075,-0.057171,0.022469,-0.096082,-0.079823,...,0.050757,0.098029,-0.009400,0.004692,0.017234,6,114.89,"[[okay, um, customer first, um, actually, we would like to pay the bills, but we told her you found out that the promotion provided the word i'm for him last december, twenty fourth of two thousan...","(VIDEO/HSD/XH, VIDEO/HSD/XH/XM)",1
6,0.292129,-0.080455,0.129113,-0.028264,0.095801,0.022306,-0.225429,0.085675,-0.137053,0.034626,...,0.082059,0.179962,-0.073717,0.175380,0.163451,6,89.00,"[[because they offered quite a bit of stuff too and, uh, to get used to keep me with contest., what could you do for me?], KEEP ME IN THE SAME PACKAGE AT A BETTER PRICE IF I WAS TO ADD M O B. I LO...",VIDEO/HSD/XM,1
7,0.005043,-0.047443,0.059971,-0.024417,0.008742,0.008770,-0.013902,0.007465,-0.079064,-0.092170,...,0.044493,0.057480,0.009285,0.027633,0.000060,0,100.00,"[[you know what?, you you're not going to be able to turn.], GOD DOESN'T HAVE A LOT OF HAVING A HUNDRED., [i'm nobody's kick out., i mean, total romance is something you heard of any other than my...",NONE,0
8,-0.015358,-0.039511,0.040222,-0.022427,-0.006896,0.003019,0.021765,0.006059,-0.057029,-0.105654,...,0.035268,0.049488,0.018356,-0.011694,-0.012719,0,0.00,,NONE,0
9,0.003669,-0.046605,0.049084,-0.016839,0.003198,-0.019879,-0.022834,0.004721,-0.096332,-0.095445,...,0.050994,0.076624,0.001243,-0.001583,-0.001749,0,24.00,"[[and i don't get it, okay?, i was just trying to do read their policy.], SO FOR YOU TO REGISTER THAT YOU HAVE TO LOOK INTO YOUR JACKLEY BROWN EMAIL EXACTLY A DECADE BROWN THIRTY FOUR., [i'm here ...",NONE,0


In [199]:
#drop some cols
final_df_with_elmo.drop(columns=["Context","Guess","Score","Price"])

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,0.002984,-0.053998,0.056018,-0.034273,0.003704,-0.004036,-0.003177,0.002186,-0.066305,-0.095743,...,-0.032543,0.036729,0.079802,0.102829,0.046919,0.047411,0.016932,0.017170,0.003122,0
1,0.047472,-0.044715,0.049914,-0.037812,0.019326,0.003511,-0.045109,0.025959,-0.092643,-0.091779,...,-0.017134,0.028860,0.062348,0.134542,0.063503,0.087361,-0.001809,0.018494,0.023757,0
2,0.042580,-0.054780,0.048289,-0.037035,0.027517,0.024342,-0.014615,0.025050,-0.067115,-0.087230,...,-0.025354,0.031925,0.045624,0.125164,0.046877,0.055976,0.008934,0.010014,0.036702,1
3,0.003881,-0.050993,0.047906,-0.026324,0.004444,0.004585,0.001779,0.013547,-0.072063,-0.104591,...,-0.031847,0.029508,0.074330,0.102445,0.034096,0.053981,0.008539,0.004612,0.002290,0
4,0.034771,-0.043803,0.040984,-0.045625,0.020423,-0.002480,-0.031445,0.055437,-0.079738,-0.082044,...,-0.014863,0.036979,0.030690,0.115476,0.043382,0.097408,-0.009731,-0.006117,0.028270,1
5,0.031361,-0.048398,0.058708,-0.041302,0.024156,-0.006075,-0.057171,0.022469,-0.096082,-0.079823,...,-0.028225,0.042479,0.044204,0.147780,0.050757,0.098029,-0.009400,0.004692,0.017234,1
6,0.292129,-0.080455,0.129113,-0.028264,0.095801,0.022306,-0.225429,0.085675,-0.137053,0.034626,...,0.020176,0.006401,-0.093407,0.332406,0.082059,0.179962,-0.073717,0.175380,0.163451,1
7,0.005043,-0.047443,0.059971,-0.024417,0.008742,0.008770,-0.013902,0.007465,-0.079064,-0.092170,...,-0.031069,0.033851,0.078443,0.117725,0.044493,0.057480,0.009285,0.027633,0.000060,0
8,-0.015358,-0.039511,0.040222,-0.022427,-0.006896,0.003019,0.021765,0.006059,-0.057029,-0.105654,...,-0.029742,0.038843,0.084521,0.076592,0.035268,0.049488,0.018356,-0.011694,-0.012719,0
9,0.003669,-0.046605,0.049084,-0.016839,0.003198,-0.019879,-0.022834,0.004721,-0.096332,-0.095445,...,-0.029511,0.032621,0.065017,0.104009,0.050994,0.076624,0.001243,-0.001583,-0.001749,0


In [200]:
xtrain, xval, ytrain, yval = train_test_split(elmo_train_new, final_df_with_elmo['label'],random_state=32, test_size=0.2)

In [203]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

model=LogisticRegression()
model.fit(xtrain, ytrain)
preds_valid = model.predict(xval)
f1_score(yval, preds_valid)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8333333333333333

In [205]:
preds_test = model.predict(xval)

In [206]:
preds_test

array([1, 1, 0, 1, 1, 1, 0, 1, 0, 1])

In [211]:
from sklearn.metrics import confusion_matrix
confusion_matrix(yval, preds_test)

array([[3, 2],
       [0, 5]])